In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras import layers, activations
from os import path
from PIL import Image
import pickle

trainingDataDir = 'o:/temp/pixiv/training/'
targetSize = (224, 224)
targetShape = (224, 224, 3)

In [2]:
def conv_block(x, filters, kernel_size=3, strides=1, padding='same'):
    x = layers.Conv2D(filters, kernel_size, strides=strides,
                      padding=padding)(x)
    x = layers.Conv2D(filters, kernel_size, strides=strides,
                      padding=padding)(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    return x


def preprocess(x):
    x = layers.RandomZoom(height_factor=0.1, width_factor=0.1)(x)
    x = layers.RandomFlip()(x)
    x = layers.RandomRotation(factor=0.1)(x)
    x = layers.Rescaling(scale=1. / 127.5,
                         offset=-1,
                         input_shape=targetShape)(x)
    return x


appModel = keras.applications.mobilenet_v2.MobileNetV2(include_top=False,
                                       weights='imagenet',
                                       input_shape=targetShape)
appModel.summary()
for layer in appModel.layers:
    layer.trainable = False


inputs = layers.Input(shape=targetShape)
x = preprocess(inputs)
x = appModel(x)
x = layers.GlobalAveragePooling2D()(x)
# x = layers.Flatten()(x)

x = layers.Dropout(0.3)(x)
x = layers.Dense(1024)(x)
x = layers.Activation(activation=activations.leaky_relu)(x)

x = layers.Dropout(0.3)(x)
x = layers.Dense(1024)(x)
x = layers.Activation(activation=activations.leaky_relu)(x)

x = layers.Dense(5, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [3]:
def get_data():
    if path.exists('ds.pickle'):
        with open('ds.pickle', 'rb') as f:
            ds = pickle.load(f)
        return ds
    else:
        df = pd.read_csv(path.join(trainingDataDir, 'data.csv'))
        imgs = []
        for i in range(len(df)):
            img = Image.open(path.join(trainingDataDir, df['img'][i]))
            xx = max(img.width, img.height)
            new_img = Image.new('RGB', (xx, xx))
            new_img.paste(img, (int(
                (xx - img.width) / 2), int((xx - img.height) / 2)))
            img = new_img.resize(targetSize, Image.BICUBIC)
            img = np.array(img)
            imgs.append(img)
        x = np.array(imgs)
        # y = np.array(df['score'].values)
        y = []
        for v in df['score'].values:
            print(v)
            row = np.zeros(5)
            row[v - 1] = 1
            y.append(row)
        y = np.array(y)
        with open('ds.pickle', 'wb') as f:
            pickle.dump((x, y), f)
        return x, y


(x, y) = get_data()


3
3
3
3
3
4
3
2
2
3
2
3
2
3
3
3
3
3
2
2
2
2
3
2
3
2
3
3
5
5
5
5
5
5
5
5
5
5
5
5
3
2
3
2
5
3
2
5
5
3
2
3
2
2
3
3
3
3
2
5
4
3
5
4
3
3
3
4
3
5
5
4
3
2
3
3
3
5
3
5
3
4
4
3
3
3
3
4
5
2
4
4
3
5
4
4
4
4
4
1
1
4
4
3
4
4
3
3
3
3
3
3
3
5
4
5
3
4
4
4
3
3
3
3
3
3
4
3
3
3
3
3
4
4
4
4
4
4
4
5
4
4
4
4
4
3
3
4
3
4
4
4
4
3
4
4
4
3
3
4
3
5
4
3
3
3
3
5
2
2
3
3
5
4
4
3
5
5
4
2
3
3
3
3
4
4
4
3
4
5
2
2
2
3
1
1
1
1
4
4
4
4
5
5
3
4
4
4
4
2
4
3
3
2
2
2
2
2
3
3
3
3
4
4
4
4
4
4
3
4
4
4
2
4
3
3
3
3
2
5
5
4
4
3
4
2
4
3
3
2
2
2
2
2
3
3
3
4
3
3
1
3
4
4
3
2
3
3
3
2
3
3
3
4
4
5
4
3
3
3
3
2
2
2
4
4
3
3
3
3
3
5
3
3
3
3
5
4
5
5
5
2
3
4
4
5
5
3
2
4
4
5
3
5
4
4
2
4
4
3
3
2
2
2
2
2
2
2
3
4
3
3
3
3
3
3
3
1
3
3
3
3
5
3
3
3
3
4
3
3
3
3
3
4
3
3
3
3
3
3
3
3
4
3
3
2
3
3
3
4
4
3
4
5
3
3
3
3
4
5
3
3
3
3
3
4
4
4
5
3
3
3
4
4
3
5
2
3
2
2
2
2
4
4
4
4
3
3
3
3
3
3
3
3
5
4
2
2
2
2
3
2
3
3
2
2
3
3
3
3
3
3
3
5
5
3
2
1
4
4
5
3
4
4
4
4
2
2
2
1
3
3
5
5
4
2
4
3
4
5
2
2
2
2
2
3
3
5
4
3
3
3
3
3
3
3
3
3
3
4
5
4
4
4
4
3
5
3
3
5
3
2
2
3
3
5
5
3
4
3


In [4]:
from matplotlib import pyplot as plt

print(model.layers[:5])
dx = model.layers[1](x[:1])
# plt.imshow(tf.reshape(tf.cast(dx, tf.uint8), targetShape))
# plt.show()
dx = model.layers[2](dx)
# plt.imshow(tf.reshape(tf.cast(dx, tf.uint8), targetShape))
# plt.show()
dx = model.layers[3](dx)
# plt.imshow(tf.reshape(tf.cast(dx, tf.uint8), targetShape))
# plt.show()
dx = model.layers[4](dx)
# plt.imshow(tf.reshape(tf.cast(dx, tf.uint8), targetShape))
# plt.show()
print(model.layers[5])
dx = model.layers[5](dx)
print(dx.shape)
print(model.layers[6])
dx = model.layers[6](dx)
print(dx.shape)
print(dx)
print(model.layers[7])
dx = model.layers[7](dx)
print(dx.shape)
print(dx)
print(model.layers[8])
dx = model.layers[8](dx)
print(dx.shape)
print(dx)
print(model.layers[9])
dx = model.layers[9](dx)
print(dx.shape)
print(dx)
dx = np.array(dx)
np.set_printoptions(threshold=np.inf)
print(dx)

[<keras.engine.input_layer.InputLayer object at 0x0000023E62118280>, <keras.layers.preprocessing.image_preprocessing.RandomZoom object at 0x0000023E61F36080>, <keras.layers.preprocessing.image_preprocessing.RandomFlip object at 0x0000023E0008B5B0>, <keras.layers.preprocessing.image_preprocessing.RandomRotation object at 0x0000023E000D3640>, <keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x0000023E3A6CD690>]
(1, 7, 7, 1280)
(1, 1280)
tf.Tensor(
[[1.0709476e-01 6.2304761e-02 1.9141276e-03 ... 1.9315526e+00
  1.2785573e-01 0.0000000e+00]], shape=(1, 1280), dtype=float32)
(1, 1280)
tf.Tensor(
[[1.0709476e-01 6.2304761e-02 1.9141276e-03 ... 1.9315526e+00
  1.2785573e-01 0.0000000e+00]], shape=(1, 1280), dtype=float32)
(1, 1024)
tf.Tensor(
[[-0.5117526  -0.02921447 -0.68583465 ... -0.30793762  0.34141183
   0.9911163 ]], shape=(1, 1024), dtype=float32)
(1, 1024)
tf.Tensor(
[[-0.10235053 -0.00584289 -0.13716693 ... -0.06158752  0.34141183
   0.9911163 ]], shape=(1, 1024),

In [5]:
cbs = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                  patience=10,
                                  verbose=1,
                                  restore_best_weights=True),
]

model.fit(
    x,
    y,
    callbacks=cbs,
    epochs=1000,
    batch_size=32,
    validation_split=0.25,
)


Epoch 1/1000
37/37 [==============================] - 5s 96ms/step - loss: 3.9461 - accuracy: 0.3995 - val_loss: 1.3631 - val_accuracy: 0.3846
Epoch 2/1000
37/37 [==============================] - 3s 69ms/step - loss: 1.3909 - accuracy: 0.4773 - val_loss: 1.7425 - val_accuracy: 0.4231
Epoch 3/1000
37/37 [==============================] - 2s 68ms/step - loss: 1.3423 - accuracy: 0.4790 - val_loss: 1.4434 - val_accuracy: 0.3590
Epoch 4/1000
37/37 [==============================] - 2s 68ms/step - loss: 1.1708 - accuracy: 0.5201 - val_loss: 2.6045 - val_accuracy: 0.3487
Epoch 5/1000
37/37 [==============================] - 2s 67ms/step - loss: 1.1559 - accuracy: 0.5577 - val_loss: 2.1178 - val_accuracy: 0.3385
Epoch 6/1000
37/37 [==============================] - 2s 68ms/step - loss: 1.0949 - accuracy: 0.5731 - val_loss: 1.6763 - val_accuracy: 0.3487
Epoch 7/1000
37/37 [==============================] - 3s 69ms/step - loss: 1.0142 - accuracy: 0.5894 - val_loss: 1.8332 - val_accuracy: 0.4282

In [7]:
def predict(img_path):
    img = Image.open(img_path)
    xx = max(img.width, img.height)
    new_img = Image.new('RGB', (xx, xx))
    new_img.paste(img, (int((xx - img.width) / 2), int((xx - img.height) / 2)))
    img = new_img.resize(targetSize, Image.BICUBIC)
    img = np.array(img)
    img = img.reshape((1, 224, 224, 3))
    return model.predict(img)


pred = predict(path.join('o:/temp/pixiv/test/', '96284448_p0.jpg'))
pred_score = np.argmax(pred) + 1
print(pred, pred_score)


[[0.01609944 0.04677213 0.6258664  0.2796855  0.03157651]] 3


In [ ]:
model.save('model.h5')